In [1]:
import webbrowser
import os
import shutil
import requests
new = 2


#open an HTML file on my own (Windows) computer
def open_html(relpath):
    url = 'file:///Users/aleks/Documents/study/phd/deeptechNikolaev/sketchfabAutoDownload/' + relpath
    # webbrowser.open(url)
    webbrowser.open(url,new=new)

# Fetching model uids

In [2]:
collection_id = '41f58a2863d9493b9ef9b3ffc95954eb'

response = requests.get('https://api.sketchfab.com/v3/collections/'+collection_id+'/models')
assert response.status_code==200
# res = response.json()['results'][0]
uids_collection = []
for res in response.json()['results']:
    uids_collection.append(res['uid'])
uids_collection

['aa63eefcd48c41d3a9d41e0a32ee6dbb',
 'be035da2e43e4f66a2405fa1508ef293',
 '493dddfd34ac4f19aa88ef6281f22459',
 '201452e568064aedadccfafb668ef6a5',
 '54d68bec471f42ac8fa6259cb8e47547',
 'd1466b9096f040278bba4acb2bc7eef3',
 'c22cd070eca748d09b222733bab358e2',
 '266a04b8716d497fab57d33eb184b0bd',
 'e161f29e8a2347e5a4ea17df8e4e6008',
 '32a93de8d7454b19bca6866746a8e3ee',
 '016f7a166eee49c080790e562f4180ce',
 '0f5e89cefe8e4c15ac58b4f9d9b49f4b',
 '9d73d65e706f4c6fa89a5d54307904e2',
 '6a63b0a50c064f2c8c0f0b2277a2363f',
 '7ebaf83ddec74174b821029e42a470f6',
 '7e8a61dd67b341e987e88299af27fe57']

# Preparing `iframe`s for `index.html`

In [3]:
with open('index_empty.html') as f:
    data = f.readlines()

In [4]:
template = [
'            <div class="row ">\n',
'                <div class="d-flex flex-column col-md-8 mb-3">\n',
'                    <iframe id="api-frame" width="auto" height="auto" allowfullscreen="true" mozallowfullscreen="true"\n',
'                        webkitallowfullscreen="true"\n',
'                        class="mb-3 rounded shadow-lg embed-responsive embed-responsive-4by3 border border-info"\n',
'                        loading="lazy"></iframe>\n',
'                </div>\n',
'            </div>\n'
]
def add_after_subs(string, substring, i):
    s_pos = string.find(substring)
    e_pos = s_pos + len(substring)
    string = string[:e_pos] + str(i) + string[e_pos:]
    return string
def alter_template(i):
    if i == 0:
        return template
    
    template_i = template.copy()
    # <div class="row ">
    template_i[0]=add_after_subs(template_i[0], 'row', i)
    # <iframe id="api-frame1" ...
    template_i[2]=add_after_subs(template_i[2], 'api-frame', i)
    return template_i

In [5]:
iframes_html = []
for i, uid in enumerate(uids_collection[:]):
    iframes_html += alter_template(i)

In [6]:
tmp=data[:14] + iframes_html + data[14:]

In [7]:
with open('index_tst.html', 'w') as the_file:
    for l in tmp:
        the_file.write(l)

In [10]:
def foo(a, b):
    return a + b
foo1 = lambda x: foo(x, 3)
foo1(4)

7

In [8]:
js_file_path = 'init_model.js'
with open(js_file_path, 'r') as file:
    # read a list of lines into data
    data = file.readlines()
#find uids row
uids_row_idx = -1
for i, l in enumerate(data):
    if 'var uids = ' in l:
        uids_row_idx = i
        break
assert uids_row_idx > 0
#alter it
data[uids_row_idx] = 'var uids = ' + str(uids_collection[:]) + ';\n'

with open('init_model.js', 'w') as file:
    file.writelines(data)